<a href="https://colab.research.google.com/github/sbhaidasna/ProductHealthSentimentAnalyst_HybridRAG/blob/main/ProductHealthSentimentAnalyst_HybridRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Product Health & Sentiment Analyst  
### Hybrid RAG Decision Primitive for Product Prioritization (v1.0)

---

## Why This Notebook Exists (PM First Principles)

One of the most critical and difficult skills of a Product Manager is **prioritization**.

In practice, prioritization is not a single signal or score. It requires **triangulating multiple dimensions**, including:

- **Customer Value**  
  Who is impacted? How severely? Are high-value customers affected?
- **Business Viability**  
  Revenue risk, contractual exposure, brand trust, and strategic alignment
- **Technical Feasibility**  
  Severity, ownership, technical debt, and effort to resolve

PMs then weight these signals differently depending on the company’s current strategic focus.

The challenge is that these signals rarely live in one place. They are fragmented across:
- long-form documentation,
- historical tracking systems,
- ownership and escalation graphs,
- and real-time customer sentiment.

This notebook explores how **agentic AI systems can support PM judgment** by reducing the cost of context aggregation, without replacing decision-making.

---

## What This Notebook Is and Is Not

### What this notebook demonstrates

This notebook implements a **decision primitive** for PM prioritization.

Specifically, it answers:

> **“Should this issue be prioritized right now, and why?”**

To do that, it:
- retrieves **semantic context** from documentation,
- retrieves **structural truth** from historical graphs,
- retrieves **fresh signals** from real-time systems,
- and synthesizes them into a **traceable, confidence-aware recommendation**.

This is a foundational building block for more advanced PM systems.

---

### What this notebook does not attempt to do

This notebook does not attempt to:
- generate a full roadmap,
- rank a portfolio of features,
- or make autonomous product decisions.

At a higher level, PMs ultimately want to ask:

> **“Given all available signals, what are the top 10 issues or features I should evaluate for my roadmap right now?”**

That portfolio-level problem requires many such decision primitives, plus additional layers for:
- strategic weighting,
- capacity constraints,
- sequencing,
- and organizational context.

This notebook intentionally focuses on **getting one critical decision right**, end to end.

---

## Core Use-Case (Decision Primitive)

### Persona: Eliza, Product Manager

**Eliza’s problem**  
Eliza needs to justify her prioritization decisions to leadership.  
She must quickly answer questions such as:

- Is this issue impacting high-value customers?
- Is it still happening right now?
- How severe is it compared to other issues?
- Can I defend this decision with evidence?

Her data is fragmented across:
- internal PRDs and design documents,
- bug tracking systems,
- ownership and escalation graphs,
- and public or community forums.

### User Story

> As a Product Manager, I want to query the impact of a feature or bug across historical and real-time systems so I can confidently prioritize work on my roadmap.

---

## Why Hybrid RAG Is the Right Reference Architecture

No single retrieval method is sufficient for PM decision-making.

### 1. Semantic RAG (Vector Search)

**What it is good at**
- Long-form narrative context
- PRDs, design rationale, and historical discussions

**What it lacks**
- Authority
- Explicit relationships
- Freshness guarantees

---

### 2. Graph RAG, Structural (Neo4j)

**What it is good at**
- Historical truth
- Ownership, escalation paths, and severity
- Multi-hop reasoning across entities

**What it lacks**
- Real-time relevance
- Qualitative nuance

---

### 3. Graph RAG, Real-Time (Memgraph)

**What it is good at**
- Fresh signals
- Low-latency sentiment and activity

**What it lacks**
- Completeness
- Authority
- Noise resistance

---

### Why PM decisions require all three

PM reasoning mirrors this structure:

> **Historical truth + Structural authority + Current reality**

This notebook uses **Hybrid RAG** to intentionally reflect how PMs already think, but at machine speed, with explicit confidence and traceability.

---

## High-Level Architecture Overview

The system executes three parallel retrieval paths, normalizes their outputs, and delegates synthesis to a single analytical authority.

| Retrieval Type | Data Source | Purpose |
|---------------|------------|---------|
| Semantic RAG | FAISS | Long-form contextual understanding |
| GraphRAG, Structural | Neo4j Aura | Bugs, ownership, escalation paths |
| GraphRAG, Real-Time | Memgraph Cloud | Fresh community sentiment and activity |

Each signal is surfaced with **freshness** and **confidence** metadata to prevent over-trust.

---

## Notebook Structure (Implementation Stages)

This notebook is organized to reflect a **real system build**, not a demo.

### Stage 1: Environment, Dependencies, and Observability
- Dependency installation
- Secure secret loading
- LangSmith tracing and observability

---

### Stage 2: Traditional RAG, Semantic Knowledge (FAISS)
- Document ingestion and chunking
- Embedding generation
- FAISS index creation
- Semantic retriever construction

**Purpose:** Provide foundational narrative context.

---

### Stage 3: Graph Substrates and Intelligence Primitives

#### Stage 3.1: Neo4j Structural Knowledge
- Graph schema definition
- Historical bug, customer, and ownership data
- Durable, authoritative relationships

**Purpose:** Answer multi-hop structural questions.

---

#### Stage 3.2: Memgraph Real-Time Data Ingestion (Production-Safe)
- Timestamped forum posts
- Deterministic sentiment relationships
- Time-window freshness guarantees

**Purpose:** Capture current user impact.

---

#### Stage 3.3: Intelligence Primitives (Claude)
- Explicit role separation  
  - Translator (Cypher compiler)  
  - Synthesizer (PM reasoning)
- Model capability guards

**Purpose:** Prevent silent model misuse.

---

#### Stage 3.4: LLM-as-a-Coder
- Natural language to Cypher translation
- Schema-aware constraints
- Read-only guarantees

**Purpose:** Enable flexible graph interrogation without hand-written queries.

---

### Stage 4: Orchestration
- Parallel retriever execution
- Context normalization
- Confidence and freshness annotation
- Final synthesis with traceability

---

### Stage 5: Evaluations
- Retriever-level inspection
- Hybrid RAG ablation testing
- Decision quality and confidence review

**Purpose:** Validate whether architecture choices materially improve PM decisions.

---

## What This Notebook Ultimately Demonstrates

- Why **architecture choices matter** for PM decision quality
- How to detect stale, missing, or conflicting signals
- When to trust which retrieval method and when not to
- How agentic AI can **augment judgment**, not replace it

This is not a toy agent.  
It is a **reference implementation for decision-centric Agentic AI systems**.

---


In [ ]:
# =============================================================
# Stage 1: ENVIRONMENT, DEPENDENCIES & OBSERVABILITY
#
# PURPOSE:
# - Create a stable, deterministic runtime
# - Install all dependencies ONCE
# - Load secrets securely
# - Enable LangSmith tracing for debugging
#
# IMPORTANT RULES:
# - No business logic
# - No LLM calls
# - No retrieval
#
# If something breaks here, the system should not proceed.
# =============================================================

# ---- Install dependencies (ONE TIME ONLY) ----
!pip install -qU \
  langchain \
  langchain-core \
  langchain-community \
  langchain-experimental \
  langchain-nvidia-ai-endpoints \
  langchain-neo4j \
  neo4j \
  pymgclient \
  faiss-cpu \
  unstructured \
  python-docx

!pip install -qU langchain-anthropic

import os
from google.colab import userdata


# ---- NVIDIA API (LLMs + Embeddings) ----
os.environ["NVIDIA_API_KEY"] = userdata.get("NVIDIA_API_KEY")

#Setting Claude API Key - This will be used for Cypher Translator & Response Synthesizer
os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")

# ---- LangSmith Observability ----
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ProductHealthSentimentAnalyst-HybridRAG"

print("Environment initialized. Dependencies installed. Tracing enabled.")



In [ ]:
# =============================================================
# Stage 2: TRADITIONAL RAG - SEMANTIC KNOWLEDGE (FAISS)
#
# PURPOSE:
# - Load unstructured documentation
# - Chunk and embed text
# - Build an in-memory FAISS index
#
# THIS IS NOT INTELLIGENCE.
# This is static semantic memory.
# =============================================================

from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ---- Configuration ----
FILE_PATH = "BMT_Internal_Documentation.docx"
EMBED_MODEL = "nvidia/nv-embedqa-e5-v5"

# ---- Embeddings ----
embeddings = NVIDIAEmbeddings(model=EMBED_MODEL)

# ---- Load document ----
loader = UnstructuredWordDocumentLoader(FILE_PATH, mode="single")
documents = loader.load()

# ---- Chunking ----
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=80,
)
chunks = splitter.split_documents(documents)

# ---- FAISS Vector Store ----
faiss_db = FAISS.from_documents(chunks, embeddings)
faiss_retriever = faiss_db.as_retriever(search_kwargs={"k": 5})

print(f"FAISS ready. Total chunks indexed: {len(chunks)}")



In [ ]:
# =============================================================
# Stage 3: GRAPH SUBSTRATES + INTELLIGENCE PRIMITIVES
# Stage 3.1: NEO4J STRUCTURAL KNOWLEDGE
#
# PURPOSE:
# - Store durable, relational product data
# - Bugs, customers, severity, escalation, ownership
#
# NO LLM LOGIC HERE.
# =============================================================

from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from google.colab import userdata

NEO4J_URI = userdata.get("NEO4J_URI")
NEO4J_PASSWORD = userdata.get("NEO4J_PASSWORD")

HISTORICAL_DATA_CYPHER = [
    "MATCH (n) DETACH DELETE n",

    "MERGE (f:Feature {name: 'Bio-Metric Tracker'})",

    "MERGE (e:Employee {name: 'Mark Olsen', title: 'VP Eng'})",
    "MATCH (f:Feature), (e:Employee) MERGE (f)-[:REVIEWED_BY]->(e)",

    "MERGE (b:Bug {id: 'BMT-CRIT-042', severity: 'Critical', initial_severity: 'Medium'})",
    "MATCH (f:Feature), (b:Bug) MERGE (f)-[:HAS_BUG]->(b)",

    "MERGE (c:Customer {id: 'CUST-001', value: 'HighValue'})",
    "MATCH (c:Customer), (b:Bug) MERGE (c)-[:REPORTED_BY]->(b)",

    "MERGE (t:Team {name: 'Team Alpha'})",
    "MATCH (b:Bug), (t:Team) MERGE (b)-[:ESCALATED_TO]->(t)",
]

def load_neo4j_data():
    driver = GraphDatabase.driver(
        NEO4J_URI, auth=("neo4j", NEO4J_PASSWORD)
    )
    with driver.session() as session:
        for stmt in HISTORICAL_DATA_CYPHER:
            session.run(stmt)
    driver.close()

load_neo4j_data()

neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username="neo4j",
    password=NEO4J_PASSWORD,
)

print("Neo4j structural data loaded.")





In [ ]:


# =============================================================
# Stage 3.2: MEMGRAPH REAL-TIME DATA INGESTION (PRODUCTION-SAFE)
#
# PURPOSE:
# - Insert forum posts with timestamps
# - Enforce exactly ONE sentiment per post
# - Prevent sentiment accumulation / row explosion
#
# DESIGN PRINCIPLES:
# - Idempotent (safe to re-run)
# - Deterministic
# - Easy to debug
# =============================================================


import time
import mgclient
from google.colab import userdata

# ---- Memgraph API ----
MEMGRAPH_HOST = userdata.get("MEMGRAPH_HOST")
MEMGRAPH_PORT = int(userdata.get("MEMGRAPH_PORT"))
MEMGRAPH_USER = userdata.get("MEMGRAPH_USER")
MEMGRAPH_PASSWORD = userdata.get("MEMGRAPH_PASSWORD")

NOW_US = int(time.time() * 1_000_000)

conn = mgclient.connect(
    host=MEMGRAPH_HOST,
    port=MEMGRAPH_PORT,
    username=MEMGRAPH_USER,
    password=MEMGRAPH_PASSWORD,
    sslmode=mgclient.MG_SSLMODE_REQUIRE,
)
cursor = conn.cursor()

# -------------------------------------------------------------
# HARD RESET SENTIMENT RELATIONSHIPS (BOTH DIRECTIONS)
# -------------------------------------------------------------
cursor.execute("""
MATCH (:ForumPost)-[r:LINKED_TO]-(:Sentiment)
DELETE r
""")

# -------------------------------------------------------------
# INGEST POSTS WITH EXACTLY ONE SENTIMENT
# -------------------------------------------------------------
posts = [
    {
        "text": "The BMT just failed again!",
        "sentiment": "Negative",
        "timestamp": NOW_US - (5 * 60 * 1_000_000),
    },
    {
        "text": "Sync issue fixed!",
        "sentiment": "Positive",
        "timestamp": NOW_US - (10 * 60 * 1_000_000),
    },
]

for post in posts:
    cursor.execute(
        """
        MERGE (fp:ForumPost {text: $text})
        SET fp.timestamp = $timestamp
        WITH fp
        MERGE (s:Sentiment {type: $sentiment})
        MERGE (fp)-[:LINKED_TO]->(s)
        """,
        {
            "text": post["text"],
            "timestamp": post["timestamp"],
            "sentiment": post["sentiment"],
        },
    )

conn.commit()
conn.close()

print("Memgraph ingestion complete (sentiment relationships normalized).")




In [ ]:
# =============================================================
# STAGE 3.3: INTELLIGENCE PRIMITIVES (CLAUDE)
#
# PURPOSE:
# - Define LLM roles with strict responsibility boundaries
# - Use only models VERIFIED to be enabled for this account
#
# ROLES:
# - Cypher LLM     → Translator / Compiler
# - Synthesis LLM  → Analyst / Decision Maker
# =============================================================

from langchain_anthropic import ChatAnthropic
import os

# -------------------------------------------------------------
# Cypher LLM (Translator / Compiler)
#
# Requirements:
# - Deterministic
# - Precise
# - Schema-faithful
# -------------------------------------------------------------

cypher_llm = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0.0,
    max_tokens=512,
)

# -------------------------------------------------------------
# Synthesis LLM (Analyst / PM Reasoner)
#
# Requirements:
# - Multi-source reasoning
# - Long context
# - Clear, grounded explanations
# -------------------------------------------------------------

synthesis_llm = ChatAnthropic(
    model="claude-3-7-sonnet-20250219",
    temperature=0.2,
    max_tokens=1200,
)

# -------------------------------------------------------------
# Sanity checks — BOTH MUST PASS
# -------------------------------------------------------------

print("Cypher LLM check:")
print(cypher_llm.invoke("Return exactly the word: OK"))

print("\nSynthesis LLM check:")
print(synthesis_llm.invoke("Respond with exactly: CHAT OK"))



In [ ]:
# =============================================================
# Stage 3.4: LLM-AS-A-CODER
#
# PURPOSE:
# - Convert natural language → Cypher
# - Act as a deterministic Cypher compiler
# - NEVER mutate the graph
# =============================================================

from langchain_core.prompts import ChatPromptTemplate

GRAPH_SCHEMA = """
You are an expert Cypher query generator.

GRAPH SCHEMA:

Nodes:
- Bug:
  - id (string, unique)
  - severity (string)
  - initial_severity (string)

- Customer:
  - id (string)
  - value (string, e.g. 'HighValue')

- Feature:
  - name (string)

- Employee:
  - name (string)
  - title (string)

- Team:
  - name (string)

Relationships:
- (:Feature)-[:HAS_BUG]->(:Bug)
- (:Customer)-[:REPORTED_BY]->(:Bug)
- (:Bug)-[:ESCALATED_TO]->(:Team)
- (:Feature)-[:REVIEWED_BY]->(:Employee)

IMPORTANT RULES:
- Always match Bug nodes using `id`, not `name`
- Respect relationship directions
- Use OPTIONAL MATCH for non-mandatory relationships
- Generate READ-ONLY queries only
- Do NOT use CREATE, MERGE, DELETE, or SET
- Return all relevant connected nodes

Return ONLY valid Cypher. No explanations.
"""

cypher_prompt = ChatPromptTemplate.from_messages([
    ("system", GRAPH_SCHEMA),
    ("human", "Question: {question}")
])

def generate_cypher_query(question: str) -> str:
    """
    Deterministically converts a natural language question
    into a READ-ONLY Cypher query.
    """
    result = (cypher_prompt | cypher_llm).invoke({"question": question})
    return result.content.strip()

print("LLM-as-a-Coder ready and schema-aligned.")


In [ ]:
# =============================================================
# STAGE 3.5 (HARDENED): CYPHER CONTRACT + VALIDATION
#
# PURPOSE:
# - Enforce deterministic Cypher structure
# - Catch invalid query shapes BEFORE execution
# - Convert DB errors into explicit system signals

""" Here's how the flow is defined
[ Stage 3.4: LLM-as-a-Coder ]
        ↓
( generates candidate Cypher )
        ↓
[ Stage 3.5: Cypher Contract + Validator ]   ← NEW
        ↓
( approved Cypher only )
        ↓
[ Stage 4 (as part of orchestration): Neo4j Execution ]"""

# =============================================================

import re

def validate_cypher(cypher: str):
    """
    Deterministic Cypher validator.
    This is NOT about correctness of results,
    but correctness of query SHAPE.
    """

    # Rule 1: Bug must be matched by id
    if "Bug" in cypher and "id:" not in cypher:
        raise ValueError(
            "Cypher contract violation: Bug nodes must be matched using `id`"
        )

    # Rule 2: WHERE after OPTIONAL MATCH requires WITH
    if "OPTIONAL MATCH" in cypher and "WHERE" in cypher:
        optional_index = cypher.find("OPTIONAL MATCH")
        where_index = cypher.find("WHERE")
        with_index = cypher.find("WITH")

        if where_index > optional_index and (with_index == -1 or with_index > where_index):
            raise ValueError(
                "Cypher contract violation: WHERE after OPTIONAL MATCH requires WITH"
            )

    # Rule 3: Disallow destructive operations
    forbidden = ["DELETE", "DETACH", "SET", "CREATE", "MERGE"]
    for word in forbidden:
        if re.search(rf"\b{word}\b", cypher, re.IGNORECASE):
            raise ValueError(
                f"Cypher contract violation: `{word}` operations are not allowed"
            )

    return True


In [ ]:
# =============================================================
# Stage 4: ORCHESTRATION
#
# PURPOSE:
# - Run all retrievers in parallel
# - Inspect and debug each retriever’s output
# - Normalize heterogeneous signals
# - Invoke synthesis LLM
#
# NO CONFIGURATION. NO MODEL CHOICES.
# =============================================================

from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter
from typing import List
import mgclient


# =============================================================
# DEBUG WRAPPER
#
# PURPOSE:
# - Print exactly what each retriever returns
# - Preserve documents unchanged for downstream steps
# =============================================================

def debug_retriever_output(name: str, docs: List[Document]) -> List[Document]:
    print("\n" + "=" * 80)
    print(f"RETRIEVER: {name}")
    print(f"DOCUMENT COUNT: {len(docs)}")

    if not docs:
        print("⚠️ WARNING: Retriever returned NO documents")

    print("-" * 80)

    for i, doc in enumerate(docs, 1):
        print(f"\nDocument {i}")
        print(f"Source: {doc.metadata.get('source')}")
        print("Content:")
        print(doc.page_content)

    print("=" * 80 + "\n")
    return docs


# =============================================================
# NEO4J RETRIEVER (STRUCTURAL) — HARDENED
#
# PURPOSE:
# - Execute ONLY validated Cypher
# - Enforce determinism outside the LLM
# - Fail safely with explicit confidence downgrade
# =============================================================

def neo4j_retriever(question: str) -> List[Document]:
    try:
        # 1. LLM proposes Cypher (Stage 3.4)
        cypher = generate_cypher_query(question)

        # 2. Deterministic enforcement (Stage 3.4a)
        validate_cypher(cypher)

        # 3. Execute only validated Cypher
        result = neo4j_graph.query(cypher)

        confidence = "high" if result else "medium"

        docs = [
            Document(
                page_content=(
                    "NEO4J STRUCTURAL DATA\n"
                    f"Cypher:\n{cypher}\n\n"
                    f"Result:\n{result}"
                ),
                metadata={
                    "source": "Neo4j",
                    "freshness": "unknown",
                    "confidence": confidence,
                },
            )
        ]

    except Exception as e:
        # 4. Fail explicitly and safely
        docs = [
            Document(
                page_content=(
                    "NEO4J STRUCTURAL DATA\n"
                    "Query failed validation or execution.\n"
                    f"Error: {str(e)}"
                ),
                metadata={
                    "source": "Neo4j",
                    "freshness": "unknown",
                    "confidence": "low",
                    "error": "cypher_validation_failure",
                },
            )
        ]

    return debug_retriever_output("Neo4j (Structural)", docs)


# =============================================================
# MEMGRAPH RETRIEVER (REAL-TIME SENTIMENT) - Checking for last 1 day of forum posts
# =============================================================

def memgraph_retriever(_: str) -> List[Document]:
    ONE_DAY_US = 24 * 60 * 60 * 1_000_000

    query = (
        "MATCH (fp:ForumPost) "
        f"WHERE fp.timestamp > (timestamp() - {ONE_DAY_US}) "
        "OPTIONAL MATCH (fp)-[:LINKED_TO]->(s:Sentiment) "
        "RETURN fp.text AS post, collect(DISTINCT s.type) AS sentiments, fp.timestamp AS ts"
    )

    conn = mgclient.connect(
        host=MEMGRAPH_HOST,
        port=MEMGRAPH_PORT,
        username=MEMGRAPH_USER,
        password=MEMGRAPH_PASSWORD,
        sslmode=mgclient.MG_SSLMODE_REQUIRE,
    )
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    conn.close()

    if not rows:
        return debug_retriever_output(
            "Memgraph (Real-Time)",
            [
                Document(
                    page_content="No forum posts in the last 1 day.",
                    metadata={
                        "source": "Memgraph",
                        "freshness": "fresh_absent",
                        "confidence": "low",
                    },
                )
            ],
        )

    # Sort newest → oldest
    rows = sorted(rows, key=lambda r: r[2], reverse=True)

    latest_post, latest_sentiments, latest_ts = rows[0]

    confidence = "high" if len(rows) >= 2 else "medium"

    docs = []

    # -------------------------------------------------
    # 1️⃣ CURRENT STATE (authoritative)
    # -------------------------------------------------
    docs.append(
        Document(
            page_content=(
                f"Current forum status: '{latest_post}' "
                f"| sentiments={latest_sentiments}"
            ),
            metadata={
                "source": "Memgraph",
                "freshness": "fresh",
                "confidence": confidence,
                "role": "current_state",
            },
        )
    )

    # -------------------------------------------------
    # 2️⃣ RECENT HISTORY (context)
    # -------------------------------------------------
    history_lines = [
        f"- {post} | sentiments={sentiments}"
        for post, sentiments, _ in rows[1:]
    ]

    if history_lines:
        docs.append(
            Document(
                page_content="Recent forum history:\n" + "\n".join(history_lines),
                metadata={
                    "source": "Memgraph",
                    "freshness": "fresh",
                    "confidence": "medium",
                    "role": "history",
                },
            )
        )

    return debug_retriever_output("Memgraph (Real-Time)", docs)


# =============================================================
# FAISS RETRIEVER (SEMANTIC)
# =============================================================

def faiss_debug_retriever(question: str) -> List[Document]:
    docs = faiss_retriever.invoke(question)

    confidence = (
        "high" if len(docs) >= 3
        else "medium" if len(docs) > 0
        else "low"
    )

    for doc in docs:
        doc.metadata.update({
            "source": "FAISS",
            "freshness": "unknown",
            "confidence": confidence,
        })

    return debug_retriever_output("FAISS (Semantic)", docs)


# =============================================================
# PARALLEL RETRIEVAL (LangSmith Instrumented)
#
# PURPOSE:
# - Run all retrievers concurrently
# - Preserve signal separation
# - Produce clean, traceable LangSmith spans
#
# Each retriever is explicitly named so that
# LangSmith traces are readable and debuggable.
# =============================================================

parallel_retrieval = RunnableParallel(
    {
        "semantic": RunnableLambda(faiss_debug_retriever).with_config(
            run_name="Retriever:FAISS:Semantic"
        ),
        "structural": RunnableLambda(neo4j_retriever).with_config(
            run_name="Retriever:Neo4j:Structural"
        ),
        "realtime": RunnableLambda(memgraph_retriever).with_config(
            run_name="Retriever:Memgraph:Realtime"
        ),
    }
).with_config(run_name="HybridRetrieval")



# =============================================================
# CONTEXT NORMALIZATION
# =============================================================

def format_context(results: dict) -> str:
    blocks = []

    for key, docs in results.items():
        for doc in docs:
            meta = doc.metadata
            header = (
                f"[{key.upper()}]\n"
                f"freshness: {meta.get('freshness')}\n"
                f"confidence: {meta.get('confidence')}\n"
            )
            blocks.append(header + "\n" + doc.page_content)

    return "\n\n".join(blocks)


# =============================================================
# FINAL PROMPT
# =============================================================

FINAL_PROMPT = ChatPromptTemplate.from_template("""
You are a Product Health & Sentiment Analyst.

Question:
{question}

Context:
{context}

Answer with clear reasoning and source attribution.
If a signal is stale, unknown, or low-confidence, explicitly state uncertainty.
If real-time signals are mixed, explicitly describe the tradeoff and explain how it affects prioritization.

At the end of your answer, include a section titled "Decision Confidence" with one of:
High, Medium, Low, or Insufficient.
Briefly justify the confidence level based on signal freshness, consistency, and coverage.

""")


# =============================================================
# FINAL CHAIN
# =============================================================

final_chain = (
    RunnablePassthrough.assign(
        context=itemgetter("question")
        | parallel_retrieval
        | format_context
    )
    | FINAL_PROMPT
    | synthesis_llm
    | StrOutputParser()
).with_config(run_name="Agent:ProductHealthDecision")

final_chain = final_chain.with_config(
    tags=["hybrid-rag", "pm-agent", "product-health"],
)

# =============================================================
# TEST
# =============================================================

print(final_chain.invoke({
    "question": "Should we prioritize BMT-CRIT-042 and why?"
}))



In [ ]:
# EVALUATIONS — PRODUCT HEALTH & SENTIMENT ANALYST

"""Goal of this section
Validate whether the Hybrid RAG system meaningfully improves PM decision-making, not just retrieval accuracy.
We focus on three PM questions that represent:
- prioritization
- customer impact
- real-time status """

# =============================================================
# Stage 5A: PM QUESTIONS (TOP 3)
# These questions act as the stable benchmark.
# Every architectural or model change should be tested against these.
# =============================================================

PM_EVAL_QUESTIONS = [
    {
        "id": "Q1",
        "question": "Should we prioritize BMT-CRIT-042 and why?",
        "intent": "End-to-end prioritization decision",
        "expected_retrievers": ["semantic", "structural", "realtime"],
    },
    {
        "id": "Q2",
        "question": "Is BMT-CRIT-042 impacting high-value customers?",
        "intent": "Customer impact and business risk",
        "expected_retrievers": ["structural", "semantic"],
    },
    {
        "id": "Q3",
        "question": "Is the BMT issue still happening right now?",
        "intent": "Freshness-sensitive status check",
        "expected_retrievers": ["realtime"],
    },
]


In [ ]:
# =============================================================
# Stage 5B: RETRIEVER-LEVEL INSPECTION - Is each retriever doing what it was designed to do?
#
# Purpose:
# - Validate each retriever independently
# - Ensure semantic(FAISS), structural(NEO4J AURA), and real-time(MEMGRAPH) signals are distinct
# - Catch silent failures early
#
# What to look for:
# - FAISS -> narrative context, not ownership
# - Neo4j -> facts, not prose
# - Memgraph -> latest signal only
#
# PM value: This explains why Hybrid RAG exists
# =============================================================

def inspect_retrievers(question: str):
    print("\n" + "=" * 100)
    print(f"QUESTION:\n{question}")

    # --- Semantic RAG (FAISS) ---
    print("\n--- Semantic RAG (FAISS) ---")
    semantic_docs = faiss_retriever.invoke(question)
    for i, d in enumerate(semantic_docs, 1):
        print(f"[{i}] {d.page_content[:200]}...")

    # --- Structural GraphRAG (Neo4j) ---
    print("\n--- Structural GraphRAG (Neo4j) ---")
    structural_docs = neo4j_retriever(question)
    for d in structural_docs:
        print(d.page_content)

    # --- Real-Time GraphRAG (Memgraph) ---
    print("\n--- Real-Time GraphRAG (Memgraph) ---")
    realtime_docs = memgraph_retriever(question)
    for d in realtime_docs:
        print(d.page_content)


# Run inspection for all top 3 questions
for q in PM_EVAL_QUESTIONS:
    inspect_retrievers(q["question"])


In [ ]:
# =============================================================
# Stage 5C: HYBRID RAG ABLATION TESTING
#
# Question:
# Does Hybrid RAG actually improve the PM decision?
#
# Method:
# - Run the SAME question with increasing signal richness
# - Compare recommendation, justification, and confidence
#
# Purpose:
# - Prove the value of Hybrid RAG
# - Surface decision deltas as retrievers are added
#
# Expected Insight:
# - Semantic only → verbose, weak authority
# - Semantic + Graph → precise, but incomplete
# - Full Hybrid → confident WITH tradeoffs
#
# PM Value:
# This is the clearest evidence that architecture choices
# materially affect decision quality.
# =============================================================

def run_ablation(question: str):
    print("\n" + "=" * 100)
    print(f"QUESTION:\n{question}")

    # Each ablation config intentionally controls
    # which signals the synthesis LLM is allowed to see.
    ablations = {
        "Semantic Only": lambda q: format_context({
            "semantic": faiss_retriever.invoke(q)
        }),
        "Semantic + Structural": lambda q: format_context({
            "semantic": faiss_retriever.invoke(q),
            # NOTE: neo4j_retriever is now hardened.
            # Validation failures degrade confidence instead of crashing.
            "structural": neo4j_retriever(q)
        }),
        "Semantic + Real-Time": lambda q: format_context({
            "semantic": faiss_retriever.invoke(q),
            "realtime": memgraph_retriever(q)
        }),
        "Full Hybrid": lambda q: format_context(
            # RunnableParallel already uses hardened retrievers
            parallel_retrieval.invoke(q)
        ),
    }

    for name, ctx_fn in ablations.items():
        print(f"\n--- {name} ---")

        try:
            # Generate context for this ablation configuration
            context = ctx_fn(question)

            # Run synthesis ONLY on retrieved context
            response = synthesis_llm.invoke(
                FINAL_PROMPT.format(
                    question=question,
                    context=context
                )
            )

            print(response.content)

        except Exception as e:
            # Ablation should never fail the notebook.
            # Any failure here is an evaluation signal.
            print(
                "[ABRATION ERROR]\n"
                "This configuration failed unexpectedly.\n"
                f"Error: {str(e)}"
            )


# Run ablation on the primary PM prioritization question
run_ablation(PM_EVAL_QUESTIONS[0]["question"])



In [ ]:
# =============================================================
# Stage 5D: DECISION QUALITY & CONFIDENCE REVIEW - Would a PM trust this output in a real roadmap meeting?
# =============================================================
# DECISION QUALITY RUBRIC
#
# Purpose:
# - Evaluate decision usefulness, not language quality
# - Penalize overconfidence more than uncertainty
#
# PM value: This aligns evals with real accountability, not synthetic benchmarks.
# =============================================================

DECISION_RUBRIC = {
    "Correctness": "Is the recommendation reasonable?",
    "Justification": "Is the 'why' grounded in evidence?",
    "Actionability": "Can this be defended to leadership?",
    "Uncertainty Handling": "Are unknowns clearly stated?",
    "Decision Confidence": "Does confidence match evidence strength?",
}

def review_decision(question: str):
    print("\n" + "=" * 100)
    print(f"QUESTION:\n{question}")

    response = final_chain.invoke({"question": question})

    print("\n--- SYSTEM RESPONSE ---\n")
    print(response)

    print("\n--- MANUAL SCORING (1–5) ---")
    for k, v in DECISION_RUBRIC.items():
        print(f"{k}: {v}")


# Review the primary decision question
review_decision(PM_EVAL_QUESTIONS[0]["question"])
